<a href="https://colab.research.google.com/github/map72ru/biz_learn/blob/main/HW3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1.  обучить несколько разных моделей на наборе данных ССЗ (train_case2.csv): 
логрег, бустинг, лес и т.д - на ваш выбор 2-3 варианта
2.  при обучении моделей обязательно использовать кроссвалидацию
3.  вывести сравнение полученных моделей по основным метрикам классификации: pr/rec/auc/f_score (можно в виде таблицы, где строки - модели, а столбцы - метрики)
4.  сделать выводы о том, какая модель справилась с задачей лучше других
5.  (опциональный вопрос) какая метрика (precision_recall_curve или roc_auc_curve) больше подходит в случае сильного дисбаланса классов? (когда объектов одного из классов намного больше чем другого).

p.s.В вопросе проще разобраться, если вспомнить оси на графике roc auc curve и рассмотреть такой пример:

Имеется 100000 объектов, из которых только 100 - класс "1" (99900 - класс "0", соответственно). Допустим, у нас две модели:

*   первая помечает 100 объектов как класс 1, но TP = 90
*   вторая помечает 1000 объектов как класс 1, но TP такой же - 90

Какая модель лучше и почему? И что позволяет легче сделать вывод - roc_auc_curve или precision_recall_curve?

6.  (опциональный вопрос) Найти порог классификации по деньгам (Стоимость лечения 15000 р, если сделали тест и начали лечить вовремя. Стоимость лечения 20000, если упустили и начали лечить когда уже проявились все симптомы. Стоимость теста 1400 р)

In [1]:
from google.colab import drive
drive.mount('/content/hw2')

Mounted at /content/hw2


In [3]:
import pandas as pd

In [4]:
data = pd.read_csv("hw2/MyDrive/data/train_case2_.csv")
print(data.shape)
data.head(3)

(70000, 13)


,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70000 entries, 0 to 69999
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           70000 non-null  int64  
 1   age          70000 non-null  int64  
 2   gender       70000 non-null  int64  
 3   height       70000 non-null  int64  
 4   weight       70000 non-null  float64
 5   ap_hi        70000 non-null  int64  
 6   ap_lo        70000 non-null  int64  
 7   cholesterol  70000 non-null  int64  
 8   gluc         70000 non-null  int64  
 9   smoke        70000 non-null  int64  
 10  alco         70000 non-null  int64  
 11  active       70000 non-null  int64  
 12  cardio       70000 non-null  int64  
dtypes: float64(1), int64(12)
memory usage: 6.9 MB


In [52]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.model_selection import cross_val_score,train_test_split,StratifiedKFold
import numpy as np
from sklearn import metrics

#import itertools

import matplotlib.pyplot as plt
from sklearn.metrics import f1_score, roc_auc_score, precision_score, classification_report, \
precision_recall_curve, confusion_matrix, recall_score

In [7]:
features = ['age', 'gender', 'height', 'weight',	'ap_hi',	'ap_lo',	'cholesterol',	'gluc',	'smoke',	'alco',	'active']
X = data[features]
y = data['cardio']

In [109]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler(copy=True, with_mean=True, with_std=True)
X_sc = scaler.fit_transform(X)
X_train,X_test,y_train,y_test=train_test_split(X_sc,y,test_size=0.25,random_state=42)

In [110]:
result = dict()

In [111]:
logreg_sc = LogisticRegression()

cv_scores = cross_val_score(logreg_sc, X_train, y_train, cv=3, scoring='roc_auc')
cv_score = np.mean(cv_scores)

# fit the model with data with cross validation
logreg_sc.fit(X_train, y_train)

y_pred=logreg_sc.predict(X_test)

r =  roc_auc_score(y_test, logreg_sc.predict_proba(X_test)[:, 1])

result['logreg'] = {'roc_auc': r, 'cv_score':  cv_score, 'Accuracy': metrics.accuracy_score(y_test, y_pred),
                    'Precision': metrics.precision_score(y_test, y_pred), "Recall": metrics.recall_score(y_test, y_pred),
                    'matrix': metrics.confusion_matrix(y_test, y_pred)}

In [112]:
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=5, random_state=42)

cv_scores = cross_val_score(clf, X_train, y_train, cv=3, scoring='roc_auc')
cv_score = np.mean(cv_scores)
print('CV score is {}'.format(cv_score))

clf.fit(X_train, y_train)

y_pred=clf.predict(X_test)
r =  roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1])
result['boosting'] = {'roc_auc': r, 'cv_score':  cv_score, 'Accuracy': metrics.accuracy_score(y_test, y_pred),
                    'Precision': metrics.precision_score(y_test, y_pred), "Recall": metrics.recall_score(y_test, y_pred),
                    'matrix': metrics.confusion_matrix(y_test, y_pred)}

CV score is 0.7533647279874719


In [113]:
clf = RandomForestClassifier(max_depth=5, random_state=42)

cv = StratifiedKFold(10)

cv_scores = cross_val_score(clf, X_train, y_train, cv=cv, scoring='roc_auc', n_jobs=8, verbose=0)
cv_score = np.mean(cv_scores)
print('CV score is {}'.format(cv_score))

clf.fit(X_train, y_train)

y_pred=clf.predict(X_test)
r =  roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1])

result['forest'] = {'roc_auc': r, 'cv_score':  cv_score, 'Accuracy': metrics.accuracy_score(y_test, y_pred),
                    'Precision': metrics.precision_score(y_test, y_pred), "Recall": metrics.recall_score(y_test, y_pred),
                    'matrix': metrics.confusion_matrix(y_test, y_pred)}



CV score is 0.7960189436582974


In [114]:

print(f'{"Метод":8} |  Precision |   Recall   |  Accurancy |  roc_auc   |  cv_score  |   Matrix')
print('----------------------------------------------------------------------------------------------------')

for classifier in result.keys():
  c = result.get(classifier)
  p = c.get('cv_score')
  r = c.get('Recall')
  a = c.get('Accuracy')
  s = c.get('cv_score')
  m = c.get('matrix')
  u = c.get('roc_auc')
  print(f'{classifier:8} | {p:.8f} | {r:.8f} | {a:.8f} | {u:.8f} | {s:.8f} | [{m[0][0]} {m[0][1]}]/[{m[1][0]} {m[1][1]}]')

Метод    |  Precision |   Recall   |  Accurancy |  roc_auc   |  cv_score  |   Matrix
----------------------------------------------------------------------------------------------------
logreg   | 0.78132816 | 0.67680436 | 0.71971429 | 0.78478682 | 0.78132816 | [6631 2057]/[2848 5964]
boosting | 0.75336473 | 0.68293236 | 0.70942857 | 0.76651423 | 0.75336473 | [6397 2291]/[2794 6018]
forest   | 0.79601894 | 0.67328643 | 0.73188571 | 0.79671429 | 0.79601894 | [6875 1813]/[2879 5933]


Результат случайного леса дает лучшие оценки, но не принципиально, Я пробовал менять параметры как кросс-валидации, так и модели, но принципиальных улучшений не получил, даже иногда модели работали хуже. Плохо то, что не дается понимания влияния тех или иных параметров на результат.

In [49]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25,random_state=42)
clf = RandomForestClassifier(max_depth=5, random_state=42)

clf.fit(X_train, y_train)

y_pred=clf.predict(X_test)
cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
cnf_matrix

array([[6875, 1813],
       [2879, 5933]])

Отдельным вопросом является кросс-валидация. В приведенной ячейке модель обучалась без кросс-валидации и видно, что результат не изменился. Я просмотрел несколько примеров из докусентации и интернета, все примерно делают одинаково. Но тогда не понятно, почему не сработала кросс-валидация? Или я что-то все-таки делаю не правильно?